In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno



In [3]:
world_bank = pd.read_csv("data/SDG_csv/SDGData.csv", sep=",")

In [4]:
life_expectancy = pd.read_csv("data/life-expectancy.csv")

In [5]:
household_ppp = pd.read_excel("data/NHA-indicators.xlsx")

In [6]:
#Zoek machine voor onderwerpen uit de indicator naam. Niet hoofdletter gevoelig. 
#Vergeet niet country_names te laden.

variabelen = []
for sentence in list(world_bank["Indicator Name"].values):
    sentence1 = np.array(sentence.split(" "))
    for words in (sentence1):
        if words.lower() == "".lower(): #Vul hier zoekterm in.
            variabelen += [sentence]

variabelen = pd.unique(variabelen)
print(variabelen)                       
                                


['Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care  (% of women age 15-49)']


In [18]:
#msno.matrix(world_bank.sample(len(world_bank)))

def find_top20():
    countries = world_bank[world_bank["Indicator Name"]== 'GDP per capita, PPP (constant 2011 international $)']["Country Name"]
    GDP_2016 = world_bank[world_bank["Indicator Name"]== 'GDP per capita, PPP (constant 2011 international $)']["2016"]
    GDP_2017 = world_bank[world_bank["Indicator Name"]== 'GDP per capita, PPP (constant 2011 international $)']["2017"]

    GDP = countries.to_frame().join(GDP_2017.to_frame()).join(GDP_2016.to_frame())
    #Mask the area's which come up at the highest and lowest countries and are undesired
    GDP = GDP.mask(GDP["Country Name"]=="Macao SAR, China")
    GDP = GDP.mask(GDP["Country Name"]=="Hong Kong SAR, China")
    GDP = GDP.mask(GDP["Country Name"]=="North America")

    lowest_20 = GDP.sort_values("2017")[0:20]
    highest_20 = GDP.sort_values("2017", ascending=False)[0:20]

    lowest_countries = lowest_20["Country Name"]
    highest_countries = highest_20["Country Name"]
    
    countries = pd.concat([lowest_countries, highest_countries])
    
    if len(lowest_countries) < 19:
        return False
    else:
        return list(countries.sort_values().values)



In [71]:
household_top20 = household_ppp
hh = pd.DataFrame()
 
    
df =household_top20[household_top20["Countries"]==find_top20()[0]]
#for country in find_top20():
    #df =household_top20[household_top20["Countries"]==country]
    
    #hh.append(household_top20[household_top20["Countries"]==country])
df[:1]

,Countries,Indicators,Unnamed: 2,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
82,Afghanistan,Household out-of-pocket payment,in current US$ per capita,NaN,NaN,13.8731,15.0541,17.6892,19.3054,21.8311,24.8895,29.1815,30.7889,36.0128,37.7654,38.8956,40.3234,43.9159,47.099
